In [107]:
from src import Processor, Reader
import pandas as pd
from datetime import datetime
from sklearn import metrics, model_selection, preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

In [108]:
# Using the US Census regions from here: https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf
south_atlantic_states = [
    10   # Delaware
    , 11 # DC
    , 12 # Florida
    , 13 # Georgia
    , 24 # Maryland
    , 37 # North Carolina
    , 45 # South Carolina
    , 51 # Virginia
    , 54 # West Virginia
]
state_filter = south_atlantic_states

In [109]:
state_dict = {
    'code' : ['10', '11', '12', '13', '24', '37', '45', '51', '54']
    , 'state_name' : ['Delware', 'DC', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'West Virginia']
}

state_code_map = pd.DataFrame(data=state_dict)

In [110]:
state_code_map.head()

,code,state_name
0,10,Delware
1,11,DC
2,12,Florida
3,13,Georgia
4,24,Maryland


## Loading data

In [111]:
# Instantiate preprocessor
preprocessor = Processor.PreProcessor(state_filter=state_filter)

In [112]:
preprocessor.load_processed_data()

/Users/ashleighlesniak/Documents/cs5644-project/src/Processor.py:36: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  merged_df = self.init_cdc_data().merge(self.init_vaccinations_data(), how='left', on=['date', 'FIPS'])
/Users/ashleighlesniak/Documents/cs5644-project/src/Processor.py:36: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  merged_df = self.init_cdc_data().merge(self.init_vaccinations_data(), how='left', on=['date', 'FIPS'])


In [113]:
# Put data into a df to work with
df = preprocessor.get_current_data()

In [79]:
df.head()

,FIPS_10001,FIPS_10003,FIPS_10005,FIPS_11001,FIPS_12001,FIPS_12003,FIPS_12005,FIPS_12007,FIPS_12009,FIPS_12011,...,restuarants_limited_open_outdoor_only_nan,restuarants_limited_open_general_indoor_Yes,restuarants_limited_open_general_indoor_nan,Series_Complete_Pop_Pct,Administered_Dose1_Pop_Pct,is_metro,Series_Complete_Pop_Pct_UR_Equity,cases,deaths,days_from_start
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,230.0,3.0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,255.0,4.0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,281.0,4.0,2
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,288.0,6.0,3
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,329.0,7.0,4


In [80]:
df.shape

(289884, 699)

In [124]:
# Export data to csv so that the data does not have to be recreated each run
df.to_csv('COVID_Analysis_Data.csv',index=False)

In [116]:
data = pd.read_csv("COVID_Analysis_Data.csv")

In [139]:
# Looking at cases and death values
#c_d_value = data[['cases','deaths']]

In [140]:
#c_d_value

In [132]:
#c_d_value.to_csv('cases_deaths_null_check.csv', index=False)

In [134]:
data.isnull().sum()

FIPS_10001                             0
FIPS_10003                             0
FIPS_10005                             0
FIPS_11001                             0
FIPS_12001                             0
                                    ... 
is_metro                               0
Series_Complete_Pop_Pct_UR_Equity      0
cases                                844
deaths                               844
days_from_start                        0
Length: 699, dtype: int64

In [138]:
null_mask = data['cases'].isnull()
null_eval = data.loc[null_mask]
null_eval

,FIPS_10001,FIPS_10003,FIPS_10005,FIPS_11001,FIPS_12001,FIPS_12003,FIPS_12005,FIPS_12007,FIPS_12009,FIPS_12011,...,restuarants_limited_open_outdoor_only_nan,restuarants_limited_open_general_indoor_Yes,restuarants_limited_open_general_indoor_nan,Series_Complete_Pop_Pct,Administered_Dose1_Pop_Pct,is_metro,Series_Complete_Pop_Pct_UR_Equity,cases,deaths,days_from_start
32984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0
32985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1
35112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0
35113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1
35114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,82
155162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,83
155163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,84
155164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,85


In [117]:
# Converting date column for filtering
data['date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

# Filtering data by date for training dataset
end_date_train = '2021-06-14'
train_mask = (data['date'] <= end_date_train)

# Training dataset with date (wd)
data_train_wd = data.loc[train_mask]

# Creating training dataset and removing date
data_train = data_train_wd.drop(columns=['date'])

In [118]:
# Filtering data by date for test dataset
start_date_test = '2021-06-15'
test_mask = (data['date'] >= start_date_test)

# Test dataset with date (wd)
data_test_wd = data.loc[test_mask]

# Creating training dataset and removing date
data_test = data_test_wd.drop(columns=['date'])

In [119]:
# Separating out columns to exclude response variables for training and testing
data_train_cols = [c for c in data_train.columns if c not in ['cases','deaths']]
data_test_cols = [c for c in data_test.columns if c not in ['cases','deaths']]

In [120]:
# Creating training predictors and response variables
X_train = data_train[data_train_cols]
y_train_cases = data_train['cases']
y_train_deaths = data_train['deaths']

# Creating testing predictors and response variables
X_test = data_test[data_test_cols]
y_test_cases = data_test['cases']
y_test_deaths = data_test['deaths']

In [121]:
# Evaluating training data
data_train.head()

,FIPS_10001,FIPS_10003,FIPS_10005,FIPS_11001,FIPS_12001,FIPS_12003,FIPS_12005,FIPS_12007,FIPS_12009,FIPS_12011,...,restuarants_limited_open_outdoor_only_nan,restuarants_limited_open_general_indoor_Yes,restuarants_limited_open_general_indoor_nan,Series_Complete_Pop_Pct,Administered_Dose1_Pop_Pct,is_metro,Series_Complete_Pop_Pct_UR_Equity,cases,deaths,days_from_start
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,230.0,3.0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,255.0,4.0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,281.0,4.0,2
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,288.0,6.0,3
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,329.0,7.0,4


In [122]:
# Evaluating the size of the training and test datasets
print(data_train.shape)
print(data_test.shape)

(253428, 698)
(36456, 698)


In [123]:
# Testing on cases

model = LinearRegression()

scores = model_selection.cross_validate(model, X_train, y_train_cases, cv=5, scoring='neg_root_mean_squared_error')
    
## Training model on entire training dataset
model.fit(X_train, y_train_cases)

y_preds = model.predict(X_test)

print()
# The mean square error
rmse = np.sqrt(np.mean((y_preds - y_test_cases) ** 2))
print(rmse)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py", line 518, in fit
    X, y = self._validate_data(X, y, accept_sparse=accept_sparse,
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 823, in check_X_y
    y = check_array(y, accept_sparse=

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').